# Ejercicio - Sistema de Recomendación usando SVD

En este ejercicio vamos a usar una una porción del [Online Retail dataset](https://archive.ics.uci.edu/ml/datasets/Online+Retail), que contiene información sobre transacciones que ocurrieron en Noviembre 2011 en una tienda online basada en el Reino Unido (UK). Esta tienda vende artículos diversos de decoración y regalos, y muchos de sus clientes son tiendas físicas.

El objetivo del ejercicio es crear un sistema de recomendación que, dado un cliente (cuyo identificador conocemos, no un nuevo cliente), recomiende productos que dicho cliente estaría interesado en comprar.

In [1]:
import pandas as pd
import numpy as np

### Cargamos los datos

In [2]:
ventas = pd.read_csv("./data/retail.csv")

In [3]:
ventas.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,573744,21314,SMALL GLASS HEART TRINKET POT,8,2011-11-01 08:16:00,2.10,17733.0,United Kingdom
1,573744,21704,BAG 250g SWIRLY MARBLES,12,2011-11-01 08:16:00,0.85,17733.0,United Kingdom
2,573744,21791,VINTAGE HEADS AND TAILS CARD GAME,12,2011-11-01 08:16:00,1.25,17733.0,United Kingdom
3,573744,21892,TRADITIONAL WOODEN CATCH CUP GAME,12,2011-11-01 08:16:00,1.25,17733.0,United Kingdom
4,573744,21915,RED HARMONICA IN BOX,12,2011-11-01 08:16:00,1.25,17733.0,United Kingdom


Creamos un diccionario para guardar las descripciones de producto

In [4]:
item_dict = dict(zip(ventas['StockCode'], ventas['Description']))

Así, dada una id de producto, podemos ver el nombre.

In [5]:
item_dict["21314"]

'SMALL GLASS HEART TRINKET POT'

### Crear matriz de Cliente/Producto (CustomerID, StockCode), donde la intersección sea el numero de veces que cada cliente ha comprado cada producto.

Este dataset no está ordenado de la forma necesaria para poder aplicar SVD (matriz de clientes/productos), asi que parte del ejercicio consiste en manipular el dataset hasta obtener la forma deseada. El identificador de cliente es `CustomerID` y el identificador de producto es `StockCode`

**Pista** Una forma de procesar el dataset para convertirlo a una matriz es con el método [`pandas.pivot_table()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html)

Queremos una matriz donde las filas sean el id del cliente `CustomerID`, las columnas sean el código de producto `StockCode`, y los valores sean la cantidad comprada de cada producto por cada cliente.

In [6]:
matriz_ventas_df = ventas.pivot_table(
    values='Quantity', 
    index='CustomerID', 
    columns='StockCode',
    aggfunc="sum"
)

In [7]:
matriz_ventas_df.head()

StockCode,10080,10120,10124A,10124G,10125,10135,11001,15030,15034,15036,...,90214M,90214N,90214S,BANK CHARGES,C2,CRUK,D,DOT,M,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12349.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12352.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
12356.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12357.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12362.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0


In [8]:
matriz_ventas_df.shape

(1711, 2704)

Tenemos una matriz de 1711 clientes y 2704 productos

In [9]:
customer_id_lista = np.array(matriz_ventas_df.index.tolist())
item_id_lista = np.array(matriz_ventas_df.columns.tolist())

In [10]:
customer_id_lista

array([ 12349.,  12352.,  12356., ...,  18274.,  18276.,  18283.])

In [11]:
item_id_lista

array(['10080', '10120', '10124A', ..., 'DOT', 'M', 'POST'],
      dtype='<U12')

Ahora creamos la matriz escasa (sparse)

In [12]:
from scipy.sparse import coo_matrix

Para poder crear la matriz sparse tenemos que eliminar los NaN, los reemplazamos por 0

In [13]:
ventas_mtz = matriz_ventas_df.fillna(0).values.copy()
ventas_mtz_sparse = coo_matrix(ventas_mtz)

In [14]:
ventas_mtz_sparse

<1711x2704 sparse matrix of type '<class 'numpy.float64'>'
	with 55529 stored elements in COOrdinate format>

## Hacer la descomposicion SVD sparse de la matriz original de cliente/producto, (podemos tomar k=10)

In [15]:
from scipy.sparse.linalg import svds

U, s, V = svds(ventas_mtz_sparse, k=10)

In [16]:
print('Tamaños')
print(f'U: {U.shape}')
print(f's: {s.shape}')
print(f'V: {V.shape}')

Tamaños
U: (1711, 10)
s: (10,)
V: (10, 2704)


# Reconstruir la matriz original de cliente/producto con SVD

In [17]:
# Convertimos el array diagonal a una matriz diagonal
s_diag = np.diag(s)

In [19]:
ventas_svd = U @ s_diag @ V

Ahora tenemos la matriz recreada pero rellenada con los valores  latentes producidos por la descomposicion SVD

In [20]:
ventas_svd.shape

(1711, 2704)

In [21]:
ventas_svd

array([[  5.35933696e-04,   1.28052995e-03,   2.01128431e-05, ...,
          5.90257550e-04,  -1.52337657e-02,   5.37978794e-02],
       [  4.85998891e-05,   1.20374209e-04,   1.71779851e-06, ...,
          5.25221195e-05,   1.44783840e-03,   4.46877775e-03],
       [  6.73127138e-06,   2.04804778e-05,   1.96671604e-07, ...,
          7.62166096e-06,  -5.78393382e-05,   1.90390914e-04],
       ..., 
       [  3.29712133e-19,   1.81773219e-18,   2.19555867e-20, ...,
         -2.71281918e-19,   2.29107452e-14,  -5.57755083e-17],
       [ -6.49041982e-08,  -1.86346004e-07,  -1.42547025e-09, ...,
         -7.07688783e-08,   6.88413885e-07,  -1.37401051e-07],
       [  1.48087850e-03,   4.31613157e-03,   4.29277782e-05, ...,
          1.75324423e-03,   3.43191936e-01,   4.95828338e-02]])

### Crear función de recomendación.

Hasta aquí hemos explicado todo en clase, ahora llega la parte adicional donde se proporciona valor a dicha decomposicion.

Ahora tenemos una matriz con las "puntuaciones" que cada cliente daria a cada producto. Lo que tenemos que hacer es crear una funcion que dado un cliente, tome la fila que le corresponde en la matriz que hemos calculado con SVD. Para dicha fila, aquellos valores más altos serán aquellos con una puntuacion estimada más alta para dicho usuario. Asi que simplemente tomamos los productos (las columnas) con valores más altos **que no estuviesen en el dataset original para el usuario!**, ya que no le debemos recomendar productos que ya ha comprado, sino productos nuevos.

Dicha función debe tomar como argumento una id de cliente y un número de recomendaciones y debe devolver las recomendaciones.

In [38]:
def recomendar(id_cliente, num_recomendaciones=5):
    # cogemos la fila de la matriz que corresponde a la id de cliente 
    cliente_index = np.where(customer_id_lista == id_cliente)[0][0]

    # Ordenamos las compras predichas por los clientes en valor descendente
    index_sort = ventas_svd[cliente_index, :].argsort()[::-1]

    # creamos una máscara booleana (True/False) de los productos que no ha comprado el cliente
    productos_no_comprados = ventas_mtz[cliente_index, :][index_sort] == 0

    rec_index = index_sort[productos_no_comprados]
    rec_ids = item_id_lista[rec_index]
    recomendaciones = rec_ids[:num_recomendaciones]
    return recomendaciones

Ahora si tenemos un cliente en concreto (por ejemplo el cliente con `id="12352"`):

In [39]:
cliente_id = 12352

In [40]:
matriz_ventas_df.head()

StockCode,10080,10120,10124A,10124G,10125,10135,11001,15030,15034,15036,...,90214M,90214N,90214S,BANK CHARGES,C2,CRUK,D,DOT,M,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12349.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12352.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
12356.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12357.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12362.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0


In [47]:
d = matriz_ventas_df.loc[cliente_id]
[item_dict[x] for x in d[d.notna()].index]

['BLUE STRIPE CERAMIC DRAWER KNOB',
 'VICTORIAN GLASS HANGING T-LIGHT',
 'IVORY KITCHEN SCALES',
 'MINT KITCHEN SCALES',
 'CHILDS BREAKFAST SET DOLLY GIRL ',
 'PINK BABY BUNTING',
 'PANTRY ROLLING PIN',
 'PANTRY PASTRY BRUSH',
 'ZINC HEART FLOWER T-LIGHT HOLDER',
 'GLASS BON BON JAR',
 'PETIT TRAY CHIC',
 'SET 12 COLOUR PENCILS SPACEBOY ',
 'SET 12 COLOUR PENCILS DOLLY GIRL ',
 'WOODLAND BUNNIES LOLLY MAKERS',
 'POSTAGE']

Podemos ver las recomendaciones con la funcion que acabamos de crear

In [50]:
recomendaciones = recomendar(cliente_id, 10)
print(recomendaciones)

['22492' '85099B' '22152' '21810' '22356' '21787' '20973' '23583' '22629'
 '22382']


In [51]:
[item_dict[x] for x in recomendaciones]

['MINI PAINT SET VINTAGE ',
 'JUMBO BAG RED RETROSPOT',
 'PLACE SETTING WHITE STAR',
 'CHRISTMAS HANGING STAR WITH BELL',
 'CHARLOTTE BAG PINK POLKADOT',
 'RAIN PONCHO RETROSPOT',
 '12 PENCIL SMALL TUBE WOODLAND',
 'LUNCH BAG PAISLEY PARK  ',
 'SPACEBOY LUNCH BOX ',
 'LUNCH BAG SPACEBOY DESIGN ']